# Models 

Want to use your favorite model? Your perfect tempreature? Me too.

In [1]:
import os
import axiomic as ax

with ax.models.Generic.Text.Large & ax.models.VerboseText:
    response = ax.infer('How much wood would a woodchuck chuck?', system_prompt='Respond in only 1 sentence.').value()

(axiomic) weave in helpful console mode (export AXIOMIC_CONSOLE_ENABLE=False to disable).

(axiomic) discovered OPENAI_API_KEY, models available under models.OpenAI

(axiomic) discovered ANTHROPIC_API_KEY, models available under models.Anthropic

(axiomic) discovered TOGETHER_API_KEY, models available under models.Together

Output()

together_text: 32 toks -> meta-llama/Llama-3-70b-chat-hf -> 26 toks (in 3.29s)

## Generic Models

Generic models will choose a reasonable model for you, based on which API keys are available in the environment. Using generic models means your code is sharable and will run for anyone else using axiomic.

The global default text model is `Generic.Text.Medium`. You can change the default to large, or a specific model such as GPT-4o or Llama or Claude. 

In [2]:
ax.models.global_default(ax.models.Generic.Text.Large & ax.models.VerboseText)
ax.infer('What is 2 + 2?', system_prompt='Respond a number.').print()

Output()

together_text: 27 toks -> meta-llama/Llama-3-70b-chat-hf -> 7 toks (in 2.59s)

/var/folders/m0/21rq1gt53f9b88p14y6q8pfm0000gn/T/ipykernel_69440/1326174865.py:2 (printed)

╭─ Text ───────────╮
│ The answer is 4. │
╰──────────────────╯

Text(_history_complete6)

## Together Example

We can use some together models specifically.

In [3]:
if 'TOGETHER_API_KEY' in os.environ:
    with ax.models.Together.Text.Llama3.llama_3_70b_chat_hf & ax.models.MaxTokens1024 & ax.models.Temperature0_9:
        ax.infer('What is your favorite color?').print()

Output()

together_text: 16 toks -> meta-llama/Llama-3-70b-chat-hf -> 65 toks (in 1.11s)

/var/folders/m0/21rq1gt53f9b88p14y6q8pfm0000gn/T/ipykernel_69440/2539479150.py:3 (printed)

╭─ Text ──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ I'm just an AI, I don't have personal preferences, including favorite colors. I am programmed to provide        │
│ information and assist with tasks, but I don't have personal opinions or feelings. However, I can tell you      │
│ about the different meanings and associations that various colors have in different cultures and contexts if    │
│ you're interested!                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## OpenAI Example

This will only work if you `export OPENAI_API_KEY=sk-...`. 

In [4]:
if 'OPENAI_API_KEY' in os.environ:
    GPT4 = ax.models.OpenAI.MultiModal.GPT4o & ax.models.Temperature0_8 & ax.models.MaxTokens128
    with GPT4:
        response = ax.infer('1 + 1 =').print()

Output()

openai_text: 12 toks -> gpt-4o -> 7 toks (in 1.06s)

/var/folders/m0/21rq1gt53f9b88p14y6q8pfm0000gn/T/ipykernel_69440/2094992855.py:4 (printed)

╭─ Text ────╮
│ 1 + 1 = 2 │
╰───────────╯

## Model Context

These contexts will properly shadow to give you fine grain control. You can always inspect your current context to understand the detailed behavior.

Here, notice the model is `RUNTIME:small_text_model` meaning it will be dynamically resolved at runtime, and can vary each time the program is run. This means that as you change your environment variables and as Axiomic updates the default models (such as when a new model is released), your agents will automatically adjust.

In [5]:
with ax.models.Generic.Text.Small & ax.models.Temperature0_1:
    with ax.models.MaxTokens4096:
        ax.models.print_context()

╭───────────────────────────────────────────────── Context Stack ─────────────────────────────────────────────────╮
│ ╭────────────────────────────────────────────── Global Context ───────────────────────────────────────────────╮ │
│ │ llm_provider_name: RUNTIME:large_text_provider                                                              │ │
│ │ llm_model_name: RUNTIME:large_text_model                                                                    │ │
│ │ llm_temperature: 0.5                                                                                        │ │
│ │ llm_max_tokens: 1024                                                                                        │ │
│ │ verbose_llm: True                                                                                           │ │
│ │ image_provider_name: together_img                                                                           │ │
│ │ image_model_name: stabilityai/stable-diffusion-2-1                                                          │ │
│ │ image_width: 1024                                                                                           │ │
│ │ image_height: 1024                                                                                          │ │
│ │ embedding_provider_name: openai_embed                                                                       │ │
│ │ embedding_model_name: text-embedding-3-small                                                                │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭──────────────────────────────────── Generic.Text.Small & Temperature0_1 ────────────────────────────────────╮ │
│ │ llm_provider_name: RUNTIME:small_text_provider                                                              │ │
│ │ llm_model_name: RUNTIME:small_text_model                                                                    │ │
│ │ llm_temperature: 0.1                                                                                        │ │
│ │ llm_max_tokens: 1024                                                                                        │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─────────────────────────────────────────────── MaxTokens4096 ───────────────────────────────────────────────╮ │
│ │ llm_max_tokens: 4096                                                                                        │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Custom Context

If you want to create a custom context, you can. Custom contexts are currently a prototype feature, but you can experiment with them.

You'll need to be careful, as it is very easy to make an invalid configuration context without reading the provider's documentation.

In [6]:
MyModel = ax.models.Config(llm_temperature=0.15, llm_max_tokens=77, _context_name="Perfect Settings")
with MyModel:
    ax.models.print_context()

╭───────────────────────────────────────────────── Context Stack ─────────────────────────────────────────────────╮
│ ╭────────────────────────────────────────────── Global Context ───────────────────────────────────────────────╮ │
│ │ llm_provider_name: RUNTIME:large_text_provider                                                              │ │
│ │ llm_model_name: RUNTIME:large_text_model                                                                    │ │
│ │ llm_temperature: 0.5                                                                                        │ │
│ │ llm_max_tokens: 1024                                                                                        │ │
│ │ verbose_llm: True                                                                                           │ │
│ │ image_provider_name: together_img                                                                           │ │
│ │ image_model_name: stabilityai/stable-diffusion-2-1                                                          │ │
│ │ image_width: 1024                                                                                           │ │
│ │ image_height: 1024                                                                                          │ │
│ │ embedding_provider_name: openai_embed                                                                       │ │
│ │ embedding_model_name: text-embedding-3-small                                                                │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── Perfect Settings ──────────────────────────────────────────────╮ │
│ │ llm_temperature: 0.15                                                                                       │ │
│ │ llm_max_tokens: 77                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯